# Análise Descritiva dos Dados 

## Base: Crime Data in Brazil

### fonte: https://www.kaggle.com/inquisitivecrow/crime-data-in-brazil

### Para começar a 'add' estou importando as bibliotecas Pandas para fazer operações de sql e Seaborn par plotar gráficos.

In [1]:
import pandas as pd
import seaborn as sns

%matplotlib inline
%config IPCompleter.greedy=True

### Instalando e em seguida importando o psycopg2 que usarei para criar a conexão com o banco de dados

In [2]:
!pip install psycopg2

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import psycopg2

In [4]:
conn = psycopg2.connect(host='localhost',database = 'criminalidade',user='postgres',password='123456')

### Primeira análise rápida usando pg_stats

#### Foram escolhidos os campos que representam o nome do atributo, fração de nulos existentes em cada atributo, número de valores distintos, valores mais comuns e frequencia dos valores mais comuns

In [5]:
resultado = pd.read_sql_query("""select attname,null_frac,n_distinct,most_common_vals,most_common_freqs from pg_stats where schemaname = 'public' """,conn)

In [6]:
resultado

,attname,null_frac,n_distinct,most_common_vals,most_common_freqs
0,id_delegacia,0.000000,328.0,"{900020,10211,10216,20249,10247,10326,10226,10...","[0.230567, 0.0201, 0.0188, 0.0178667, 0.015766..."
1,nome_departamento,0.000000,18.0,"{""DECAP "",""D...","[0.732533, 0.230567, 0.0212333]"
2,nome_seccional,0.000000,58.0,"{""DELEGACIA ELETRONICA "",""D...","[0.230567, 0.116033, 0.115467, 0.104633, 0.085..."
3,nome_delegacia,0.000000,328.0,"{""DELEGACIA ELETRONICA "",""11º D.P. SA...","[0.230567, 0.0201, 0.0188, 0.0178667, 0.015766..."
4,cidade,0.000000,12.0,"{""S.PAULO ""}",[0.998533]
5,ano_bo,0.000000,5.0,"{2016,2014,2013,2015,2017}","[0.325867, 0.235767, 0.211867, 0.198467, 0.028..."
6,num_bo,0.000000,25439.0,"{17,308,55,108,329,579,600,1338,1738,35,122,15...","[0.0004, 0.0004, 0.000366667, 0.000366667, 0.0..."
7,nome_departamento_circ,0.000000,1.0,"{""DECAP ""}",[1.0]
8,nome_seccional_circ,0.000000,8.0,"{""DEL.SEC.3º OESTE "",""D...","[0.172833, 0.153033, 0.143333, 0.1311, 0.11226..."
9,nome_delegacia_circ,0.000000,94.0,"{""01º D.P. SE "",""03º D.P. CA...","[0.0302, 0.0242, 0.02, 0.0192333, 0.0189, 0.01..."


####  Ao olhar o campo null_frac, aparentemente vemos uma base sem nenhum valor null, exceto no atributo logradouro onde tem baixíssimo indice de nulos(0.0033% da base), porém olhando com uma maior atenção encontraremos alguns problemas, se observar o atributo datahora_ocorrencia_bo nos campos most_common_vals e most_common_freq, verá que o valor mais comum não é null e sim uma String 'NULL' e além disso a frequencia é 1, ou seja, esse atributo é completamente inválido e deverá ser removido do banco, pois não agrega nenhum valor ás nossas análises.


### Dropando a coluna datahora_ocorrencia_bo

In [7]:
resultado2 = pd.read_sql_query("""alter table crime drop column datahora_comunicacao_bo """,conn)

TypeError: 'NoneType' object is not iterable

#### Em um segundo olhar, logo vemos que mais atributos tem a String 'NULL' como uns dos valores mais comuns, então agora investigaremos alguns desses atributos de forma um pouco mais minuciosa, começando pelo último atributo cor_cutis.

In [8]:
resultado3 = pd.read_sql_query("""select cor_cutis,count(*) from crime group by cor_cutis order by count(*) desc""",conn)

In [9]:
resultado3

,cor_cutis,count
0,Branca,794943
1,NULL,454486
2,Parda,267873
3,Preta,49322
4,Outros,20692
5,Amarela,14081
6,Vermelha,482
7,25,9
8,24,8
9,34,8


#### Com esse teste rapidamente percebemos 3 coisas, primeira de aproximadamente 1.602.000 registros, 2 em cor_cutis são realmente nulos, segunda os campos verdadeiramente nulos estão com a String 'NULL' e somam cerca de 400 mil registros, terceira apareceram vários números, logo, não são cores da pele. Vamos tentar descobrir a causa disso, vamos seleconar os dados onde cor_cutis é igual aos números.

In [10]:
resultado4 = pd.read_sql_query("""select * from crime where not (cor_cutis like '%Branca%') and not (cor_cutis like '%Parda%') and cor_cutis not like '%Amarela%' and cor_cutis not like '%Vermelha%' and cor_cutis not like '%Preta%'  and cor_cutis not like '%Outros%' and cor_cutis is not null and cor_cutis not like '%NULL%'

""",conn)

In [11]:
resultado4

,id_delegacia,nome_departamento,nome_seccional,nome_delegacia,cidade,ano_bo,num_bo,nome_departamento_circ,nome_seccional_circ,nome_delegacia_circ,...,descr_subtipolocal,logradouro,numero_logradouro,latitude,longitude,descr_tipo_pessoa,flag_vitima_fatal,sexo_pessoa,idade_pessoa,cor_cutis
0,10004,DIRD - DEPTO IDENT.REG.DIV,DIV.POL.PORTO/AERO/PROT.TURIS-DECADE,06º D.P. METROPOLITANO,S.PAULO,2013,302,DECAP,DEL.SEC.1º CENTRO,01º D.P. SE,...,Metrov. e ferroviário metrop.-Desembarque ...,ENTRE BRAS/SÉ,DESEMBARQUE,999,-23.54938522,-46.63325723,Vítima,NULL,M,43
1,10004,DIRD - DEPTO IDENT.REG.DIV,DIV.POL.PORTO/AERO/PROT.TURIS-DECADE,06º D.P. METROPOLITANO,S.PAULO,2015,900016,DECAP,DEL.SEC.5º LESTE,30º D.P. TATUAPE,...,Metrov. e ferroviário metrop.-outros ...,ESTAÇÃO TATUAPÉ,MEZANINO AREA LIVRE,99,NULL,NULL,Vítima,NULL,M,55
2,10004,DIRD - DEPTO IDENT.REG.DIV,DIV.POL.PORTO/AERO/PROT.TURIS-DECADE,06º D.P. METROPOLITANO,S.PAULO,2015,900016,DECAP,DEL.SEC.5º LESTE,30º D.P. TATUAPE,...,Metrov. e ferroviário metrop.-outros ...,ESTAÇÃO TATUAPÉ,MEZANINO AREA LIVRE,99,NULL,NULL,Vítima,NULL,F,25
3,10217,DECAP,DEL.SEC.2º SUL,17º D.P. DOUTOR ALDO GALIANO,S.PAULO,2015,3505,DECAP,DEL.SEC.2º SUL,17º D.P. DOUTOR ALDO GALIANO,...,Via pública ...,RUA ISAURA ABAD GIACOMELLI,,87,NULL,NULL,Vítima,NULL,M,25
4,10217,DECAP,DEL.SEC.2º SUL,17º D.P. DOUTOR ALDO GALIANO,S.PAULO,2015,3505,DECAP,DEL.SEC.2º SUL,17º D.P. DOUTOR ALDO GALIANO,...,Via pública ...,RUA ISAURA ABAD GIACOMELLI,,87,NULL,NULL,Vítima,NULL,M,24
5,10226,DECAP,DEL.SEC.2º SUL,26º D.P. SACOMA,S.PAULO,2015,6687,DECAP,DEL.SEC.2º SUL,95º D.P. HELIÓPOLIS,...,Vidraria-Acesso,/Escada/Elevador/Passarela ...,AVENIDA ALMIRANTE DELAMARE,800,-23.60736817,-46.59262144,Vítima,NULL,F,51
6,10226,DECAP,DEL.SEC.2º SUL,26º D.P. SACOMA,S.PAULO,2015,6687,DECAP,DEL.SEC.2º SUL,95º D.P. HELIÓPOLIS,...,Vidraria-Acesso,/Escada/Elevador/Passarela ...,AVENIDA ALMIRANTE DELAMARE,800,-23.60736817,-46.59262144,Vítima,NULL,M,47
7,10226,DECAP,DEL.SEC.2º SUL,26º D.P. SACOMA,S.PAULO,2015,6687,DECAP,DEL.SEC.2º SUL,95º D.P. HELIÓPOLIS,...,Vidraria-Acesso,/Escada/Elevador/Passarela ...,AVENIDA ALMIRANTE DELAMARE,800,-23.60736817,-46.59262144,Vítima,NULL,F,79
8,10004,DIRD - DEPTO IDENT.REG.DIV,DIV.POL.PORTO/AERO/PROT.TURIS-DECADE,06º D.P. METROPOLITANO,S.PAULO,2013,2816,DECAP,DEL.SEC.1º CENTRO,06º D.P. CAMBUCI,...,Metrov. e ferroviário metrop.-vagão ...,ENTRE ESTA§ÃO S.JOAQUIM,LIBERDADE,999,-23.57806782,-46.6405597,Vítima,NULL,F,30
9,10248,DECAP,DEL.SEC.6º SANTO AMARO,48º D.P. CIDADE DUTRA,S.PAULO,2013,1372,DECAP,DEL.SEC.6º SANTO AMARO,101º D.P. JDIM IMBUIAS,...,Serralheria-Acesso,/Escada/Elevador/Passarela ...,R DR OSCAR ANDRADE LEMOS,910,-23.73392446,-46.69437827,Vítima,NULL,M,56


#### Perceba que a linha por algum motivo deu um SHIFT na hora do ETL e corrompeu esses dados, ou seja, a idade está em cor_cutis, como achamos esse resultado no último atributo, todas as linhas afetadas por esse shift estão presentes, logo apagaremos todas essas linhas da nossa base de dados, juntamente com as linhas onde 'cor_cutis' = "NULL".

In [14]:
resultado5 = pd.read_sql_query("""delete from crime where not (cor_cutis like '%Branca%') and not (cor_cutis like '%Parda%') and cor_cutis not like '%Amarela%' and cor_cutis not like '%Vermelha%' and cor_cutis not like '%Preta%'  and cor_cutis not like '%Outros%'""",conn)

TypeError: 'NoneType' object is not iterable

In [15]:
resultado5

NameError: name 'resultado5' is not defined

#### Agora vamos analisar outras colunas que requerem uma formatação especial, por exemplo, a coluna hora_ocorrencia_bo que registra a hora da ocorrencia de um bo e está em um formato txt. 

In [21]:
resultado6 = pd.read_sql_query("""select hora_ocorrencia_bo,count(*) from crime group by hora_ocorrencia_bo order by count(*) desc""",conn)

In [22]:
resultado6

,hora_ocorrencia_bo,count
0,NULL,436156
1,20:00,27572
2,21:00,27108
3,22:00,25592
4,19:00,24374
5,20:30,22071
6,23:00,21563
7,14:00,20973
8,15:00,20405
9,21:30,20059


#### Ao final do resultado6, observamos que aparecem alguns dados em formato impróprio para a manipulação no banco de dados, agora faremos uma verificação se são os unicos casos em que isso ocorre ou se existem outros casos com mais número de registros onde isso ocorre, ou seja, onde hora_ocorrencia_bo é diferente de String 'NULL' e diferente de formato 12:00.

In [29]:
resultado7 = pd.read_sql_query("""select hora_ocorrencia_bo from crime where hora_ocorrencia_bo not like '%:%' and hora_ocorrencia_bo not like 'NULL'""",conn)

In [30]:
resultado7

,hora_ocorrencia_bo
0,15H50
1,15H50
2,1910
3,23H00
4,23H00


#### Como existem poucos dados que foram inseridos com uma formatação inadequada, faremos 3 alterações para mudar os valores desse atributo para o formato correto

In [33]:
pd.read_sql_query("""update crime set hora_ocorrencia_bo = '15:50' where hora_ocorrencia_bo = '15H50' """,conn)

TypeError: 'NoneType' object is not iterable

In [ ]:
resultado8

In [17]:
resultado6 = pd.read_sql_query("""select hora_ocorrencia_bo from crime where hora_ocorrencia_bo like '%:%' """,conn)

In [19]:
resultado6

,hora_ocorrencia_bo
0,06:20
1,06:10
2,05:30
3,12:05
4,05:50
5,23:20
6,06:45
7,14:30
8,20:00
9,05:30


In [ ]:
resultado['hora_ocorrencia_bo'] = pd.to_datetime(resultado['hora_ocorrencia_bo'])

In [ ]:
resultado['hora_ocorrencia_bo'] = pd.to_datetime(resultado['hora_ocorrencia_bo'], format= '%H:%M').dt.time

## Formatando o horario

In [ ]:
resultado